### 데이터 받아오기

In [1]:
import json
import numpy as np
import random
import nltk

In [2]:
with open("reviews_Books_5.json","r") as data_file:
    raw_data=data_file.readlines()

In [3]:
json_list = []
for i in range(len(raw_data)):
    try:
        json_data = json.loads(raw_data[i])
        json_list.append(json_data)
    except:
        print(raw_data[i],i)

{"reviewerID": "AAXUNK0W2DZG5", "asin": "0060520841", "reviewerName": "Amazon Customer \"leneker\"", "helpful": [5, 10], "reviewText": "1996 Bernard Goldberg wrote an editorial for the Wall Street Journal that said there was an obvious bias on the part of network new shows  for the liberal point of view.  he then illustrated this with an example that he dissected in-depth.  The reaction to this observation was the ruination of her career, and the beginning of his status as a pariah among most newsmen.  This book is used to add more ammo to the controversy.That the journalists who so eagerly pry into other peoples lives and business should be reluctant to be examined is hardly surprising.  Almost no one really wants to have his life or business dissected by Mike Wallace not even Dan Rather.  Some facts in this book are really potent such as the survey results which show how the average journalist and the average American are often vastly at odds.  Other chapters highlight different stor

### 데이터 분리하기

In [4]:
review_data = []

for i in range(len(json_list)):
    review_dict={}
    review_dict["reviewText"] = json_list[i]["reviewText"]
    review_dict["overall"] = json_list[i]["overall"]
    review_data.append(review_dict)

In [5]:
reviewText = []
for i in range(len(review_data)):
    reviewText.append(review_data[i]["reviewText"])

In [6]:
reviewText[9]

"An amazing work. Realizing extensive use of Biblical imagery and sentence structure, &#34;The Prophet&#34; by Khalil Gibran is a literary classic. Influencing the Free Love movement of the 1960's, Gibran's master work explores themes of love, longing and loss."

In [7]:
y_data = []
for i in range(len(review_data)):
    y_data.append(review_data[i]["overall"])

In [8]:
from collections import Counter
Counter(y_data)

Counter({1.0: 4239, 2.0: 5102, 3.0: 10177, 4.0: 21555, 5.0: 52111})

<h3>데이터 전처리</h3>

In [9]:
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re

In [10]:
def review_to_words(raw_review):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    
   
    cleaned_text_list=[]
    for i in range(len(raw_review)):
        
        review_text = BeautifulSoup(raw_review[i]).get_text()
        letters_only = re.sub("[^a-zA-Z]", " ", review_text)
        words = letters_only.lower().split()
        cleaned_text = " ".join(words)
        cleaned_text_list.append(cleaned_text)

    return cleaned_text_list

       

In [11]:
pre_data=review_to_words(reviewText)

c:\users\jangy\miniconda3\envs\bi_project\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file c:\users\jangy\miniconda3\envs\bi_project\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


In [12]:
print(len(reviewText), len(pre_data), len(y_data))

93184 93184 93184


### 데이터 셔플

In [13]:
c = list(zip(pre_data, y_data))
random.shuffle(c)

In [14]:
a, b = zip(*c)

In [15]:
pre_text_data=list(a)
overall_data=np.array(list(b))

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word', sublinear_tf=True,lowercase=True)
vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf = vectorizer.fit_transform(pre_text_data)

### 가장 작은 평점으로 갯수 맞추기

In [17]:
from collections import Counter
length=sorted(Counter(overall_data).most_common())[2][1]

In [18]:
# pre_data=[]
# overall_data=[]
# for j in range(len(reviewText)):
#     try:
#         if 50<=len(reviewText[j]) or len(reviewText[j])<=1500:
#             pre_data.append(reviewText[j])
#             overall_data.append(y_data[j])
#     except:
#         print(j)

In [19]:
search=[1.0,2.0,3.0,4.0,5.0]
count=0
text=[]
y=[]

for i in search:
    count=0
    for index,value in enumerate(overall_data):
        if (count==length*(1/2)): break
        elif (i==value):
            text.append(pre_text_data[index])
            y.append(value)
            count=count+1

In [20]:
Counter(y)

Counter({1.0: 4239, 2.0: 5102, 3.0: 10177, 4.0: 21555, 5.0: 52111})

In [21]:
print(len(text), len(y))

93184 93184


<h3>데이터 셔플 다시</h3>

In [22]:
c = list(zip(text, y))
random.shuffle(c)

In [23]:
a, b = zip(*c)

In [24]:
text_data=list(a)
overall=np.array(list(b))

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word', sublinear_tf=True,lowercase=True, stop_words='english', ngram_range=(1,2))
# vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=2,max_df=0.2,stop_words='english')

tfidf = vectorizer.fit_transform(text_data)

# from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer(analyzer='word', sublinear_tf=True,ngram_range=(1,2),lowercase=True,min_df=2,max_df=0.2,stop_words='english')
# tfidf = vectorizer.fit_transform(text_data)

In [26]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(tfidf , overall , test_size=0.33, random_state=100)

In [27]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((62433, 3530578), (62433,), (30751, 3530578), (30751,))

### 예측

#### Multiclass SVM

In [28]:
# from sklearn import datasets
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn.svm import LinearSVC

# result = OneVsRestClassifier(LinearSVC(random_state=100)).fit(x_train, y_train)
# result = result.predict(x_test)

In [29]:
# from sklearn.metrics import accuracy_score
# accuracy_score(result ,y_test)

In [30]:
# from sklearn.metrics import classification_report
# print(classification_report(y_test, result))

### LogisticRegression

In [31]:
from sklearn import linear_model
logreg = linear_model.LogisticRegression()

logreg.fit(x_train, y_train)
logreg.predict(x_test)==y_test

# from sklearn import linear_model
# #logreg = linear_model.LogisticRegression()
# logreg = linear_model.LogisticRegression(random_state=42,solver='sag',multi_class='multinomial',warm_start=True)
# logreg.fit(x_train, y_train)

array([False,  True,  True, ..., False,  True, False], dtype=bool)

In [32]:
print(y_test[:10], logreg.predict(x_test)[:10])

[ 3.  5.  5.  5.  5.  4.  5.  3.  5.  5.] [ 5.  5.  5.  5.  5.  4.  5.  4.  5.  4.]


In [33]:
from sklearn.metrics import accuracy_score
accuracy_score(logreg.predict(x_test), y_test)


0.60632174563428831

In [34]:
from sklearn.metrics import precision_score
precision_score(logreg.predict(x_test), y_test,average=None) 

array([ 0.10373134,  0.04750594,  0.11869266,  0.21358543,  0.96420843])

In [35]:
from sklearn import metrics
for i in range(1,6):
    fpr, tpr, thresholds = metrics.roc_curve(y_test, logreg.predict(x_test), pos_label=i)
#pos_label Label considered as positive and others are considered negative.
    print(i,"점 = ",metrics.auc(fpr, tpr))

1 점 =  0.391982718416
2 점 =  0.351762303686
3 점 =  0.35320971045
4 점 =  0.457838818895
5 점 =  0.639565380162


In [36]:
result=logreg.predict(x_test)

In [37]:
from sklearn.metrics import classification_report
print(classification_report(y_test, result))

             precision    recall  f1-score   support

        1.0       0.66      0.10      0.18      1340
        2.0       0.46      0.05      0.09      1684
        3.0       0.42      0.12      0.19      3488
        4.0       0.43      0.21      0.29      7140
        5.0       0.64      0.96      0.77     17099

avg / total       0.56      0.61      0.53     30751



### INPUT 

In [38]:
texts = ["""I enjoyed this book, as long as I kept in mind that the book was written for young adults.  
         The characters were believable and it was easy to identify with their pain.""",
        "I think the author gets lost in his own thoughts and trys to make storys of something that's not there.",
        """I was very excited to start reading this book and although I did like it, 
        I thought that a lot of the words to describe body parts was kind of high-schoolish.  
        Didn't care for the cliff hanging ending.""",
        """I don't know if you are familiar with FANFIC on the internet but that what this book reminded me of.  
        I love the series and thought I would like the books...not so much.""",
        """As I love western era romances, this one was wonderful.  An easy read and ended the way we always hope....happily!"""]
overall=[4,1,3,2,5]
vecs = vectorizer.transform(review_to_words(texts)) 

c:\users\jangy\miniconda3\envs\bi_project\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file c:\users\jangy\miniconda3\envs\bi_project\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


In [39]:
print(overall,logreg.predict(vecs))

[4, 1, 3, 2, 5] [ 5.  5.  3.  5.  5.]
